## From video, we can check the crop image.

In [ ]:
from utils.video_utils import extract_frames_fast

# --- CONFIGURATION ---
VIDEO_PATH    = '/media/holidayj/Documents/Data/Platform/Chungmuro/chungmuro_hasun_20221019_f1729_t2030/chungmuro_hasun_20221019T172940_20221019T203040.mp4'
OUTPUT_FOLDER = 'frames/chungmuro_hasun_20221019_f1729_t2030_1frame_700'

INTERVAL_SEC  = 1/3   # Extract 3 frames per second (approx)
CROP_SIZE     = 700
MARGIN_RIGHT  = 400
MARGIN_TOP    = 30
FILE_PREFIX   = "chungmuro_frame"

def main():
    extract_frames_fast(
        video_path=VIDEO_PATH,
        output_folder=OUTPUT_FOLDER,
        interval_sec=INTERVAL_SEC,
        crop_size=CROP_SIZE,
        margin_right=MARGIN_RIGHT,
        margin_top=MARGIN_TOP,
        file_prefix=FILE_PREFIX
    )

if __name__ == '__main__':
    main()

'/media/holidayj/Documents/github/ML/Python/annotation'

## From hd images, crop images.

In [1]:
import cv2
import os
import glob
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# --- Configuration ---
SOURCE_FOLDER = "/media/holidayj/Documents/Data/Platform/Chungmuro/chungmuro_hasun_20221019_f1729_t2030/frames/chungmuro_hasun_10frame_1920_train_arrival"
OUTPUT_FOLDER = "./cropped_images_700"

# Crop Parameters
CROP_SIZE = 700
MARGIN_RIGHT = 400
MARGIN_TOP = 30

def save_image_worker(args):
    """
    Independent worker function to save the image.
    (Reused from utils/video_utils.py pattern)
    """
    img_data, save_path = args
    cv2.imwrite(save_path, img_data)

def process_existing_images(source_folder, output_folder, crop_size, margin_right, margin_top):
    # 1. Setup
    if not os.path.exists(source_folder):
        print(f"Error: Source folder not found at {source_folder}")
        return

    os.makedirs(output_folder, exist_ok=True)

    # Get list of images (assuming .jpg, add .png if needed)
    image_paths = sorted(glob.glob(os.path.join(source_folder, "*.jpg")))
    
    if not image_paths:
        print("No .jpg images found in source folder.")
        return

    # 2. Calculate Crop Coordinates based on the first image
    first_img = cv2.imread(image_paths[0])
    if first_img is None:
        print("Error reading the first image.")
        return

    h, width, _ = first_img.shape
    
    # Logic: x_start = width - margin_right - crop_size
    x_start = width - margin_right - crop_size
    y_start = margin_top
    
    # Boundary checks
    if x_start < 0: x_start = 0
    if y_start < 0: y_start = 0

    print(f"--- Processing {len(image_paths)} Images ---")
    print(f"Original Size: {width}x{h}")
    print(f"Crop: {crop_size}x{crop_size} at ({x_start}, {y_start})")
    
    # 3. Initialize Worker Pool
    worker_count = max(1, cpu_count() - 1)
    print(f"Using {worker_count} background processes for saving.")
    
    pool = Pool(processes=worker_count)
    saved_count = 0

    # 4. Processing Loop
    print("Starting batch crop...")
    
    for img_path in tqdm(image_paths, unit="img"):
        # Read image in main process
        frame = cv2.imread(img_path)
        
        if frame is None:
            continue

        # Crop Logic
        cropped = frame[y_start : y_start + crop_size, 
                        x_start : x_start + crop_size]
        
        # Construct filename (keep original name)
        filename = os.path.basename(img_path)
        save_path = os.path.join(output_folder, filename)
        
        # Async Save
        pool.apply_async(save_image_worker, args=((cropped, save_path),))
        saved_count += 1

    # 5. Cleanup
    print("\nProcessing finished. Waiting for file writes to complete...")
    pool.close()
    pool.join()
    print(f"Done! Saved {saved_count} images to '{output_folder}'.")

if __name__ == "__main__":
    process_existing_images(SOURCE_FOLDER, OUTPUT_FOLDER, CROP_SIZE, MARGIN_RIGHT, MARGIN_TOP)

--- Processing 4161 Images ---
Original Size: 1920x1080
Crop: 700x700 at (820, 30)
Using 7 background processes for saving.
Starting batch crop...


100%|██████████| 4161/4161 [02:11<00:00, 31.58img/s]


Processing finished. Waiting for file writes to complete...
Done! Saved 4161 images to './cropped_images_700'.


Scanning reference directory...
Found 4161 images in Source.
Found 2769 files in Reference.
Starting copy process...


  0%|          | 0/4161 [00:00<?, ?file/s]

100%|██████████| 4161/4161 [00:00<00:00, 4760.19file/s]


--- Summary ---
Total processed: 4161
Copied to TEMP1 (Matched): 1384
Copied to TEMP2 (Rest):    2777
Done.


## Saving 1 frame.


## Finding Cropping area from Video

In [ ]:
import cv2
import os

# --- CONFIGURATION ---
video_path      = '/media/holidayj/Documents/Data/Platform/Chungmuro/chungmuro_sangsun_20221019_f1729_t2029/chungmuro_sangsun_20221019T172940-20221019T202940.mp4'
output_folder   = 'output_frames'
output_filename = 'cropped_700.jpg'
# crop_size       = 600
crop_size       = 700

# Cropping Margins
margin_top   = 30    # Move down pixels from the top edge
margin_right = 400  # Move left 120 pixels from the right edge
# ---------------------

os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video at {video_path}")
else:
    # 1. Search for the first valid frame (Fix for the [h264] error)
    frame_found = False
    max_attempts = 100
    
    print("Searching for a valid Keyframe...")
    
    for i in range(max_attempts):
        ret, frame = cap.read()
        if ret:
            print(f"Success: Valid frame found at index {i}")
            
            # --- CROP LOGIC STARTS HERE ---
            
            # 2. Get Dimensions
            height, width, _ = frame.shape
            
            # 3. Calculate Coordinates
            # Y: Start at top margin
            y_start = margin_top
            y_end = y_start + crop_size

            # X: Start from right side (width) - margin - crop_size
            x_end = width - margin_right
            x_start = x_end - crop_size

            print(f"Original Resolution: {width}x{height}")
            print(f"Cropping Area -> X: {x_start} to {x_end}, Y: {y_start} to {y_end}")

            # 4. Perform Crop
            cropped_frame = frame[y_start:y_end, x_start:x_end]

            # 5. Save
            full_save_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(full_save_path, cropped_frame)
            print(f"Saved cropped image to: {full_save_path}")
            
            frame_found = True
            break # Stop after saving the first valid frame
            
            # --- CROP LOGIC ENDS HERE ---

    if not frame_found:
        print("Error: Could not find any valid frames in the beginning of the video.")

cap.release()

Searching for a valid Keyframe...
Success: Valid frame found at index 1
Original Resolution: 1920x1080
Cropping Area -> X: 1280 to 1920, Y: 0 to 640
Saved cropped image to: output_frames/cropped_700.jpg


[h264 @ 0x3a1d3240] missing picture in access unit with size 40
[h264 @ 0x3a1d3240] no frame!
[h264 @ 0x3a0c2180] no frame!


# Cropping area from the full frame images

In [4]:
import os
import cv2
from tqdm import tqdm  # Optional: for a progress bar, run 'pip install tqdm' if missing

# 1. Configuration
source_dir = "/media/holidayj/Documents/Data/Platform/Chungmuro/chungmuro_sangsun_20221019_f1729_t2029/chungmuro_sangsun_10frames_1920_train_arrival"
output_dir = os.path.join(source_dir, "cropped")
CROP_W, CROP_H = 640, 640

# 2. Setup
os.makedirs(output_dir, exist_ok=True)
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')

# Get list of images
files = [f for f in os.listdir(source_dir) if f.lower().endswith(image_extensions)]
print(f"Found {len(files)} images. Processing...")

# 3. Processing Loop
count = 0
for filename in tqdm(files):
    file_path = os.path.join(source_dir, filename)
    
    # Read Image
    img = cv2.imread(file_path)
    if img is None:
        print(f"Warning: Could not read {filename}")
        continue
    
    h, w, _ = img.shape
    
    # Check if image is large enough
    if w < CROP_W or h < CROP_H:
        print(f"Skipping {filename}: Image smaller than crop size ({w}x{h})")
        continue

    # 4. Calculate Top-Right Coordinates
    # Y: Starts at 0, ends at 640
    # X: Starts at (Width - 640), ends at Width
    x_start = w - CROP_W
    y_start = 0
    
    # Crop: img[y:y+h, x:x+w]
    cropped_img = img[y_start : y_start + CROP_H, x_start : x_start + CROP_W]
    
    # 5. Save
    save_path = os.path.join(output_dir, filename)
    cv2.imwrite(save_path, cropped_img)
    count += 1

print(f"\nDone! {count} images saved to:\n{output_dir}")

Found 4945 images. Processing...


100%|██████████| 4945/4945 [02:52<00:00, 28.66it/s]


Done! 4945 images saved to:
/media/holidayj/Documents/Data/Platform/Chungmuro/chungmuro_sangsun_20221019_f1729_t2029/chungmuro_sangsun_10frames_1920_train_arrival/cropped


## Put cropped images into set1 to set5 folders. (Round Robin)

In [6]:
import os
import shutil

# 1. Configuration
base_dir = "/media/holidayj/Documents/Data/Platform/Chungmuro/chungmuro_sangsun_20221019_f1729_t2029/chungmuro_sangsun_10frames_1920_train_arrival/cropped"
num_sets = 5

# 2. Get and Sort Files
valid_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
files = [f for f in os.listdir(base_dir) if f.lower().endswith(valid_exts)]
files.sort() # Important to keep the sequence (1st, 2nd, 3rd...)

print(f"Found {len(files)} images. Distributing cyclically into {num_sets} sets...")

# Create the set folders first
for i in range(1, num_sets + 1):
    os.makedirs(os.path.join(base_dir, f"set{i}"), exist_ok=True)

# 3. Distribute Files Round-Robin
for index, filename in enumerate(files):
    # Calculate which set (0 to 4) -> (1 to 5)
    # 0 % 5 = 0 -> set1
    # 1 % 5 = 1 -> set2
    # ...
    # 5 % 5 = 0 -> set1
    set_num = (index % num_sets) + 1
    
    src_path = os.path.join(base_dir, filename)
    dst_path = os.path.join(base_dir, f"set{set_num}", filename)
    
    shutil.move(src_path, dst_path)

print("Done! Distribution complete.")

Found 4945 images. Distributing cyclically into 5 sets...
Done! Distribution complete.


## Extracting full frames

In [ ]:
import cv2
import os
import math
from datetime import datetime, timedelta
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# --- CONFIGURATION ---
VIDEO_PATH    = '/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/4_2020-11-28_14-21-05.mp4'
OUTPUT_FOLDER = '/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/extracted_frames_10frame'

# Video Start Time (Hour, Minute, Second)
START_TIME_STR = "14:21:05"

FRAME_STEP    = 3
CROP_SIZE     = 320
MARGIN_RIGHT  = 120
MARGIN_TOP    = 0
# ---------------------

def extract_frames_worker(args):
    """
    Worker function to be run by each CPU core.
    Now accepts 'fps' and 'start_time_obj' to calculate timestamps.
    """
    video_path, frames_to_process, (dir_crop, dir_orig), (x_start, y_start), fps, start_dt = args
    
    cap = cv2.VideoCapture(video_path)
    count = 0
    
    for frame_idx in frames_to_process:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        
        if ret:
            # --- Calculate Timestamp ---
            # Seconds elapsed = frame_number / fps
            seconds_elapsed = frame_idx / fps
            
            # Add to start time
            current_time = start_dt + timedelta(seconds=seconds_elapsed)
            
            # Format: HHMMSS (e.g., 070001)
            time_str = current_time.strftime("%H%M%S")
            
            # New Filename: euljiro_070001_frame_000030.jpg
            filename = f"euljiro_{time_str}_frame_{frame_idx:06d}.jpg"

            # 1. Save Original
            path_orig = os.path.join(dir_orig, filename)
            cv2.imwrite(path_orig, frame)

            # 2. Save Cropped
            cropped = frame[y_start : y_start + CROP_SIZE, 
                            x_start : x_start + CROP_SIZE]
            
            path_crop = os.path.join(dir_crop, filename)
            cv2.imwrite(path_crop, cropped)
            count += 1
    
    cap.release()
    return count

def main():
    # 1. Setup Folders
    dir_crop = os.path.join(OUTPUT_FOLDER, 'cropped')
    dir_orig = os.path.join(OUTPUT_FOLDER, 'original')
    os.makedirs(dir_crop, exist_ok=True)
    os.makedirs(dir_orig, exist_ok=True)

    # 2. Parse Start Time
    # We use a dummy date (today) because timedelta requires a datetime object
    start_dt = datetime.strptime(START_TIME_STR, "%H:%M:%S")

    # 3. Analyze Video Metadata
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print(f"Error: Cannot open video at {VIDEO_PATH}")
        return

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    cap.release()

    # 4. Config
    x_start = width - MARGIN_RIGHT - CROP_SIZE
    y_start = MARGIN_TOP
    
    print(f"Video FPS: {fps}")
    print(f"Start Time: {START_TIME_STR}")
    print(f"Total Frames: {total_frames}")
    print(f"Saving to: {OUTPUT_FOLDER}")

    # 5. Generate Target Indices (Every 10 frames)
    target_indices = list(range(0, total_frames, FRAME_STEP))
    
    print(f"Extracting {len(target_indices)} frames (Step: {FRAME_STEP}) using {cpu_count()} CPUs...")

    # 6. Distribute work
    num_cpus = cpu_count()
    chunk_size = math.ceil(len(target_indices) / num_cpus)
    
    tasks = []
    for i in range(0, len(target_indices), chunk_size):
        chunk = target_indices[i : i + chunk_size]
        # Pass fps and start_dt to worker
        tasks.append((VIDEO_PATH, chunk, (dir_crop, dir_orig), (x_start, y_start), fps, start_dt))

    # 7. Execute
    with Pool(processes=num_cpus) as pool:
        with tqdm(total=len(target_indices), unit="img") as pbar:
            for saved_count in pool.imap_unordered(extract_frames_worker, tasks):
                pbar.update(saved_count)

    print("Done! Files saved with timestamp in name (e.g., euljiro_070001_frame_xxxxxx.jpg)")

if __name__ == '__main__':
    main()

## Extracting frames and crop

In [ ]:
'''
# chungmuro hasun config.
VIDEO_PATH    = '/media/holidayj/Documents/Videos/videos/platform/euljiro/euljoro_20251111_070000.mp4'
OUTPUT_FOLDER = '/media/holidayj/Documents/data/frames/euljiro_rush_20251111'
INTERVAL_SEC  = 0.2
CROP_SIZE     = 600
MARGIN_RIGHT  = 400
MARGIN_TOP    = 10
'''

import cv2
import numpy as np
import os
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# --- CONFIGURATION ---
VIDEO_PATH    = '/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20221101_f1700_t2000/euljiro_inner_20221101_f1700_t2000.mp4'
# VIDEO_PATH    = '/media/holidayj/Documents/Data/Platform/Chungmuro/chungmuro_hasun_20221019_f1729_t2030/chungmuro_hasun_20221019T172940_20221019T203040.mp4'
# VIDEO_PATH    = '/home/holidayj/Videos/videos/platform/chungmuro/chungmuro_sangsun_20221019T172940-20221019T202940/chungmuro_sangsun_20221019T172940-20221019T202940.mp4'

OUTPUT_FOLDER = 'frames/euljiro_inner_20221101_f1700_t2000_1sec'
INTERVAL_SEC  = 1/3
CROP_SIZE     = 700
MARGIN_RIGHT  = 400
MARGIN_TOP    = 30



# OUTPUT_FOLDER = 'frames/chungmuro_hasun_6frames_700'
# INTERVAL_SEC  = 0.2
# CROP_SIZE     = 700
# MARGIN_RIGHT  = 400
# MARGIN_TOP    = 30
# ---------------------

def save_image_worker(args):
    """
    Independent worker function to save the image.
    This runs on separate CPUs.
    """
    img_data, save_path = args
    cv2.imwrite(save_path, img_data)

def main():
    # 1. Setup
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    # 2. Metadata
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = np.round(cap.get(cv2.CAP_PROP_FPS))
    # print("fps =", np.round(fps))
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    
    # 3. Crop Config
    x_start = width - MARGIN_RIGHT - CROP_SIZE
    y_start = MARGIN_TOP
    frame_step = int(fps * INTERVAL_SEC)
    if frame_step < 1: frame_step = 1

    print(f"FPS: {fps} | Step: {frame_step}")
    print(f"Using {cpu_count()} CPUs for saving images.")

    # 4. Initialize the Worker Pool (For saving only)
    # We use roughly 80% of CPUs to leave room for the main reader process
    worker_count = max(1, cpu_count() - 1) 
    pool = Pool(processes=worker_count)
    
    current_idx = 0
    saved_count = 0

    # 5. Fast Reader Loop
    # The main loop now NEVER waits for disk I/O. 
    # It just throws the image to the pool and immediately reads the next one.
    with tqdm(total=total_frames, unit="frame") as pbar:
        while True:
            ret, frame = cap.read()

            if not ret:
                if current_idx < 100: # Skip initial corruption
                    current_idx += 1
                    pbar.update(1)
                    continue
                else:
                    break

            if current_idx % frame_step == 0:
                # Crop
                cropped = frame[0:1080,
                                0:1920]
                cropped = frame[y_start : y_start + CROP_SIZE, 
                                x_start : x_start + CROP_SIZE]
                
                # Construct path
                filename = f"chungmuro_frame_{current_idx:06d}.jpg"
                save_path = os.path.join(OUTPUT_FOLDER, filename)
                
                # --- ASYNC SAVE ---
                # Fire and forget. The main loop continues immediately.
                pool.apply_async(save_image_worker, args=((cropped, save_path),))
                saved_count += 1

            current_idx += 1
            pbar.update(1)

    cap.release()
    
    print("\nReading finished. Waiting for remaining file writes to complete...")
    pool.close()
    pool.join() # Wait for the background workers to finish saving
    print(f"Done! Saved {saved_count} images.")

if __name__ == '__main__':
    main()

# This code select frames only those divisible by 30, and crops to get the dataset.

In [1]:
import cv2
import os
import glob
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# --- CONFIGURATION ---
SOURCE_FOLDER = '/media/holidayj/Documents/github/ML/Python/annotation/frames/chungmuro_hasun_10frame_1920_train_arrival'
OUTPUT_FOLDER = os.path.join(SOURCE_FOLDER, '30_frames_crop')

# Filter Condition: Every 30 frames (0, 30, 60, 90...)
TARGET_FRAME_STEP = 30

# Crop Configuration
CROP_SIZE     = 700   # 700x700 square
MARGIN_RIGHT  = 400
MARGIN_TOP    = 30
# ---------------------

def crop_worker(args):
    """
    Worker function to read an image, crop it, and save it.
    args: (file_path, save_path, crop_coords)
    crop_coords: (y_start, y_end, x_start, x_end)
    """
    file_path, save_path, (y_s, y_e, x_s, x_e) = args
    
    img = cv2.imread(file_path)
    if img is None:
        return False

    # Crop the image using numpy slicing [y:y+h, x:x+w]
    cropped_img = img[y_s:y_e, x_s:x_e]
    
    cv2.imwrite(save_path, cropped_img)
    return True

def main():
    # 1. Setup Folders
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    
    # 2. Get list of all images
    print(f"Scanning files in: {SOURCE_FOLDER}")
    all_files = glob.glob(os.path.join(SOURCE_FOLDER, "*.jpg"))
    
    if not all_files:
        print("Error: No images found in source folder.")
        return

    # 3. Calculate Crop Coordinates (Based on the first image found)
    # We assume all images have the same resolution (likely 1920x1080)
    sample_img = cv2.imread(all_files[0])
    img_h, img_w = sample_img.shape[:2]
    
    # Logic: Start X = Width - Margin_Right - Crop_Size
    x_start = img_w - MARGIN_RIGHT - CROP_SIZE
    x_end   = x_start + CROP_SIZE
    y_start = MARGIN_TOP
    y_end   = y_start + CROP_SIZE
    
    crop_coords = (y_start, y_end, x_start, x_end)
    
    print(f"Image Size: {img_w}x{img_h}")
    print(f"Crop X: {x_start} ~ {x_end} (Width: {CROP_SIZE})")
    print(f"Crop Y: {y_start} ~ {y_end} (Height: {CROP_SIZE})")

    # 4. Filter files: Only keep frames where number % 30 == 0
    tasks = []
    print(f"Filtering for every {TARGET_FRAME_STEP}th frame...")
    
    for file_path in all_files:
        filename = os.path.basename(file_path)
        
        # Parse frame number from "chungmuro_frame_002060.jpg"
        try:
            # Split by '_' take last part, remove .jpg extension
            frame_part = filename.split('_')[-1] 
            frame_str = frame_part.split('.')[0]
            frame_num = int(frame_str)
            
            # CHECK CONDITION
            if frame_num % TARGET_FRAME_STEP == 0:
                save_path = os.path.join(OUTPUT_FOLDER, filename)
                tasks.append((file_path, save_path, crop_coords))
                
        except ValueError:
            # Skip files that don't match the naming pattern
            continue

    print(f"Found {len(tasks)} frames to process.")
    
    # 5. Execute Parallel Processing
    num_cpus = cpu_count()
    print(f"Processing with {num_cpus} CPUs...")
    
    with Pool(processes=num_cpus) as pool:
        # Use imap to show progress bar
        list(tqdm(pool.imap(crop_worker, tasks), total=len(tasks), unit="img"))

    print(f"\nSuccess! Cropped images saved to: {OUTPUT_FOLDER}")

if __name__ == '__main__':
    main()

Scanning files in: /media/holidayj/Documents/github/ML/Python/annotation/frames/chungmuro_hasun_10frame_1920_train_arrival
Image Size: 1920x1080
Crop X: 820 ~ 1520 (Width: 700)
Crop Y: 30 ~ 730 (Height: 700)
Filtering for every 30th frame...
Found 1384 frames to process.
Processing with 8 CPUs...


100%|██████████| 1384/1384 [00:15<00:00, 90.24img/s] 


Success! Cropped images saved to: /media/holidayj/Documents/github/ML/Python/annotation/frames/chungmuro_hasun_10frame_1920_train_arrival/30_frames_crop


# Applying CLAHE
Too much noise? Decrease CLIP_LIMIT from 3.0 to 2.0.

Still too dark? Increase CLIP_LIMIT to 4.0 or 5.0.

In [3]:
import cv2
import os
import glob
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# --- CONFIGURATION ---
# The folder containing the already cropped images
INPUT_FOLDER = '/media/holidayj/Documents/github/ML/Python/annotation/frames/chungmuro_hasun_10frame_1920_train_arrival/30_frames_crop'

# The new subfolder for CLAHE images
OUTPUT_FOLDER = os.path.join(INPUT_FOLDER, 'clahe')

# CLAHE Settings
# clipLimit: Higher = more contrast (and more noise). 2.0 to 4.0 is standard.
# tileGridSize: Size of the local area to inspect. (8,8) is standard.
CLIP_LIMIT = 5.0 
GRID_SIZE = (8, 8)
# ---------------------

def clahe_worker(args):
    """
    Reads an image, applies CLAHE to the Lightness channel, and saves it.
    """
    file_path, save_path = args
    
    img = cv2.imread(file_path)
    if img is None:
        return False

    # 1. Convert BGR to LAB color space
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # 2. Split into L, A, B channels
    l_channel, a, b = cv2.split(lab)

    # 3. Apply CLAHE to L-channel
    clahe = cv2.createCLAHE(clipLimit=CLIP_LIMIT, tileGridSize=GRID_SIZE)
    cl = clahe.apply(l_channel)

    # 4. Merge the CLAHE enhanced L-channel with the original A and B channels
    merged_lab = cv2.merge((cl, a, b))

    # 5. Convert back to BGR
    final_img = cv2.cvtColor(merged_lab, cv2.COLOR_LAB2BGR)
    
    cv2.imwrite(save_path, final_img)
    return True

def main():
    # 1. Setup Folders
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    
    # 2. Get list of cropped images
    print(f"Scanning files in: {INPUT_FOLDER}")
    all_files = glob.glob(os.path.join(INPUT_FOLDER, "*.jpg"))
    
    if not all_files:
        print("Error: No images found. Make sure you ran the crop script first.")
        return

    print(f"Found {len(all_files)} images. Applying CLAHE (ClipLimit={CLIP_LIMIT})...")

    # 3. Prepare Tasks
    tasks = []
    for file_path in all_files:
        filename = os.path.basename(file_path)
        save_path = os.path.join(OUTPUT_FOLDER, filename)
        tasks.append((file_path, save_path))
    
    # 4. Execute Parallel Processing
    num_cpus = cpu_count()
    print(f"Processing with {num_cpus} CPUs...")
    
    with Pool(processes=num_cpus) as pool:
        list(tqdm(pool.imap(clahe_worker, tasks), total=len(tasks), unit="img"))

    print(f"\nDone! Enhanced images saved to: {OUTPUT_FOLDER}")

if __name__ == '__main__':
    main()

Scanning files in: /media/holidayj/Documents/github/ML/Python/annotation/frames/chungmuro_hasun_10frame_1920_train_arrival/30_frames_crop
Found 1384 images. Applying CLAHE (ClipLimit=5.0)...
Processing with 8 CPUs...


100%|██████████| 1384/1384 [00:30<00:00, 45.89img/s]



Done! Enhanced images saved to: /media/holidayj/Documents/github/ML/Python/annotation/frames/chungmuro_hasun_10frame_1920_train_arrival/30_frames_crop/clahe


## 폴더안의 프레임수 csv 파일로 추출

In [63]:
import os
import pandas as pd

def create_interval_csv(source_dir, output_csv="frame_intervals_checked.csv"):
    if not os.path.exists(source_dir):
        print(f"에러: 경로를 찾을 수 없습니다 -> {source_dir}")
        return

    data = []

    # 1. 파일 목록 읽기 및 파싱
    files = [f for f in os.listdir(source_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    print(f"총 {len(files)}개의 이미지 파일을 발견했습니다. 처리를 시작합니다...")

    for filename in files:
        try:
            name_body = os.path.splitext(filename)[0]
            parts = name_body.split('_')
            
            if len(parts) >= 2:
                video_id = int(parts[0])
                frame_num = int(parts[1])
                
                data.append({
                    'video_id': video_id, 
                    'frame': frame_num,
                    'filename': filename 
                })
        except ValueError:
            continue

    if not data:
        print("처리할 데이터가 없습니다.")
        return

    # 2. DataFrame 생성 및 정렬
    df = pd.DataFrame(data)
    df = df.sort_values(by=['video_id', 'frame'])

    # 3. Interval 계산 (현재 - 이전)
    df['interval'] = df.groupby('video_id')['frame'].diff().fillna(0).astype(int)

    # 4. Next Interval 계산 (다음 - 현재)
    # 다음 프레임이 얼마나 떨어져 있는지 확인하기 위해 shift(-1) 사용
    df['next_interval'] = df.groupby('video_id')['frame'].shift(-1) - df['frame']

    # 5. "2_sec" vs "less" 라벨링 로직 적용
    # 상태 의존적(이전 프레임의 결과가 현재에 영향)이므로 순회하며 처리
    
    labeled_data = []
    
    # 비디오 ID별로 그룹화하여 처리 (비디오가 바뀌면 상태 리셋)
    for vid, group in df.groupby('video_id'):
        # 좁은 간격(30프레임 등)이 시작되었는지 추적하는 플래그
        # True면 직전 프레임이 "좁은 간격의 시작(1st)"이었음을 의미
        in_short_gap_sequence = False 
        
        for idx, row in group.iterrows():
            next_val = row['next_interval']
            
            # 마지막 프레임 처리 (next_interval이 NaN)
            if pd.isna(next_val):
                # 마지막 프레임은 다음 구간이 없으므로, 보통 유지하거나 종료 처리
                # 여기서는 'less'로 처리하거나 필요시 '2_sec'으로 변경 가능
                row['interval_type'] = '2_sec' 
            
            # Case A: 다음 프레임까지 간격이 충분함 (>35)
            elif next_val > 35:
                row['interval_type'] = '2_sec'
                in_short_gap_sequence = False # 시퀀스 초기화
            
            # Case B: 다음 프레임까지 간격이 좁음 (<=35, 예: 30)
            else:
                if in_short_gap_sequence:
                    # 이미 좁은 간격이 시작된 상태에서 또 좁은 간격 등장 -> 2번째 프레임 (제거 대상)
                    row['interval_type'] = 'less'
                    in_short_gap_sequence = False # 하나 건너뛰었으므로 다시 리셋 (다음 30은 다시 2_sec가 됨)
                else:
                    # 좁은 간격의 첫 번째 프레임 -> 유지
                    row['interval_type'] = '2_sec'
                    in_short_gap_sequence = True # 플래그 세팅

            labeled_data.append(row)

    # 결과 데이터프레임 생성
    df_final = pd.DataFrame(labeled_data)

    # 6. CSV 저장
    # 보기 편하게 컬럼 순서 정리
    output_cols = ['video_id', 'frame', 'interval', 'next_interval', 'interval_type']
    df_final[output_cols].to_csv(output_csv, index=False)

    print(f"\n완료! '{output_csv}' 파일이 생성되었습니다.")
    
    # 결과 검증 출력
    print("\n--- 결과 샘플 (30프레임 구간 확인) ---")
    # next_interval이 35 이하인 구간을 찾아 패턴 확인
    sample = df_final[df_final['next_interval'] <= 35].head(6)
    if not sample.empty:
        print(sample[output_cols])
    else:
        print(df_final[output_cols].head())

    print("\n--- Type 분포 ---")
    print(df_final['interval_type'].value_counts())

# 경로 설정
target_dir = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/Euljiro_inner_20201128_f1038_t1519/other_tries/new_annotation_skku_30_frames_ing/train_balanced_orig/TrainVal_unbalanced'

if __name__ == "__main__":
    create_interval_csv(target_dir)

총 5523개의 이미지 파일을 발견했습니다. 처리를 시작합니다...

완료! 'frame_intervals_checked.csv' 파일이 생성되었습니다.

--- 결과 샘플 (30프레임 구간 확인) ---
      video_id  frame  interval  next_interval interval_type
774          1   7080         0           30.0         2_sec
3923         1   7110        30           30.0          less
2974         1   7140        30           30.0         2_sec
4731         1   7170        30           30.0          less
3641         1   7200        30           30.0         2_sec
727          1   7230        30           30.0          less

--- Type 분포 ---
interval_type
2_sec    3801
less     1722
Name: count, dtype: int64


In [64]:
import os
import shutil
import pandas as pd

def move_2sec_frames(source_dir, subfolder_name="selected_2sec"):
    if not os.path.exists(source_dir):
        print(f"에러: 경로를 찾을 수 없습니다 -> {source_dir}")
        return

    # 이동할 타겟 폴더 생성
    target_dir = os.path.join(source_dir, subfolder_name)
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    data = []

    # 1. 파일 리스트업
    files = [f for f in os.listdir(source_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"총 {len(files)}개의 이미지 파일을 분석합니다...")

    for filename in files:
        try:
            name_body = os.path.splitext(filename)[0]
            parts = name_body.split('_')
            if len(parts) >= 2:
                video_id = int(parts[0])
                frame_num = int(parts[1])
                data.append({'video_id': video_id, 'frame': frame_num, 'filename': filename})
        except ValueError:
            continue

    if not data:
        print("데이터가 없습니다.")
        return

    # 2. 로직 적용 (이전 코드와 동일)
    df = pd.DataFrame(data)
    df = df.sort_values(by=['video_id', 'frame'])

    # 다음 프레임과의 간격 계산
    df['next_interval'] = df.groupby('video_id')['frame'].shift(-1) - df['frame']

    # 2_sec / less 분류
    labeled_data = []
    for vid, group in df.groupby('video_id'):
        in_short_gap_sequence = False 
        for idx, row in group.iterrows():
            next_val = row['next_interval']
            
            # 마지막 프레임이거나 간격이 충분하면 2_sec (유지)
            if pd.isna(next_val) or next_val > 35:
                row['interval_type'] = '2_sec'
                in_short_gap_sequence = False
            else:
                # 간격이 좁음 (<=35)
                if in_short_gap_sequence:
                    row['interval_type'] = 'less' # 이미 좁은 간격 시작됨 -> 제거 대상
                    in_short_gap_sequence = False
                else:
                    row['interval_type'] = '2_sec' # 좁은 간격의 시작 -> 유지
                    in_short_gap_sequence = True
            
            labeled_data.append(row)

    df_final = pd.DataFrame(labeled_data)

    # 3. 파일 이동 실행
    move_count = 0
    
    # 2_sec로 분류된 파일만 필터링
    files_to_move = df_final[df_final['interval_type'] == '2_sec']

    print(f"\n이동 시작: {len(files_to_move)}개의 '2_sec' 프레임 세트를 '{subfolder_name}' 폴더로 이동합니다.")

    for _, row in files_to_move.iterrows():
        img_filename = row['filename']
        base_name = os.path.splitext(img_filename)[0]
        
        src_img_path = os.path.join(source_dir, img_filename)
        dst_img_path = os.path.join(target_dir, img_filename)
        
        # 1) 이미지 이동
        if os.path.exists(src_img_path):
            shutil.move(src_img_path, dst_img_path)
        
        # 2) 텍스트 파일(.txt) 이동
        txt_filename = base_name + ".txt"
        src_txt_path = os.path.join(source_dir, txt_filename)
        dst_txt_path = os.path.join(target_dir, txt_filename)
        
        if os.path.exists(src_txt_path):
            shutil.move(src_txt_path, dst_txt_path)
            
        move_count += 1

    print(f"\n작업 완료! 총 {move_count}세트의 파일이 이동되었습니다.")
    print(f"남은 파일(less)은 원래 폴더에 있습니다.")

# 경로 설정
target_dir = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/Euljiro_inner_20201128_f1038_t1519/other_tries/new_annotation_skku_30_frames_ing/train_balanced_orig/TrainVal_unbalanced'

if __name__ == "__main__":
    move_2sec_frames(target_dir)

총 5523개의 이미지 파일을 분석합니다...

이동 시작: 3801개의 '2_sec' 프레임 세트를 'selected_2sec' 폴더로 이동합니다.

작업 완료! 총 3801세트의 파일이 이동되었습니다.
남은 파일(less)은 원래 폴더에 있습니다.


# Counting class

In [15]:
import os
import glob
from collections import defaultdict

# Define the path to your dataset


## Euljiro peak and off-peak combined datasets
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_3_Original_dataset_Euljiro_peak_n_offpeak_combined/test/am_peak'

# pm peak
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_1_Original_dataset_Eljiro_peak/Euljiro_inner_20221101_f1700_t2000_120_interval_till_1830/val'
# AM peak
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_1_Original_dataset_Eljiro_peak/Euljiro_inner_20251111_f0700_t1000_120_frames_interval/val'
# off-peak
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/3_class/Euljiro_off_peak_Test'
dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/3_class/Euljiro_off_peak_TrainVal_balanced_2sec_plus_addition_1sec_FINAL_20260117/train'


# # off-peak test balanced
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/3_class/Euljiro_off_peak_Test'
# # off-peak train balanced
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/3_class/Euljiro_off_peak_TrainVal_balanced_2sec_plus_addition_1sec_FINAL_20260117/train'
# # off-rush val balanced
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/3_class/Euljiro_off_peak_TrainVal_balanced_2sec_plus_addition_1sec_FINAL_20260117/val'
# off-peak TrainVal balanced
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/3_class/Euljiro_off_peak_TrainVal_balanced_2sec_plus_addition_1sec_FINAL_20260117/TrainVal'

# # off-peak train  unbalanced
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/Euljiro_off_peak_TrainVal_unbalanced_2sec_FINAL_20260117/train'
# # off-rush val unbalanced
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/Euljiro_off_peak_TrainVal_unbalanced_2sec_FINAL_20260117/val'
# off-peak TrainVal unbalanced
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/Euljiro_off_peak_TrainVal_unbalanced_2sec_FINAL_20260117/TrainVal'

# peak test
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_1_Original_dataset_Eljiro_peak/Euljiro_inner_20221101_f1700_t2000_120_interval_till_1830/test'
# peak train
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_1_Original_dataset_Eljiro_peak/Euljiro_inner_20221101_f1700_t2000_120_interval_till_1830/train'
# peak val
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_1_Original_dataset_Eljiro_peak/Euljiro_inner_20221101_f1700_t2000_120_interval_till_1830/val' 





# Chungmuro dataset
# sangsun
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/chungmuro/chungmuro_sangsun_20221019_f1729_t2029/train'
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/chungmuro/chungmuro_sangsun_20221019_f1729_t2029/val'
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/chungmuro/chungmuro_sangsun_20221019_f1729_t2029/test'

# hasun
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/chungmuro/chungmuro_hasun_20221019_f1729_t2030/train'
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/chungmuro/chungmuro_hasun_20221019_f1729_t2030/val'
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/chungmuro/chungmuro_hasun_20221019_f1729_t2030/test'

# 2 class combined off-peak train balanced
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/2_class/Euljiro_off_peak_Test'
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/2_class/Euljiro_off_peak_Test'
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/2_class/Euljiro_off_peak_Test'
# dataset_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/2_class/Euljiro_off_peak_Test'


# Find all .txt files in the directory
label_files = glob.glob(os.path.join(dataset_path, '*.txt'))

print(f"Found {len(label_files)} label files in {dataset_path}")

# Initialize a dictionary to count objects per class
class_counts = defaultdict(int)

# Iterate through each label file
for file_path in label_files:
    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                # Ensure the line is not empty
                if parts:
                    # In YOLO format, the first element is the class ID
                    class_id = int(parts[0])
                    class_counts[class_id] += 1
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

# Print the results
print("\nObject counts per class:")
# Sort by class ID for cleaner output
for class_id in sorted(class_counts.keys()):
    print(f"Class {class_id}: {class_counts[class_id]}")

Found 4882 label files in /media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/3_class/Euljiro_off_peak_TrainVal_balanced_2sec_plus_addition_1sec_FINAL_20260117/train

Object counts per class:
Class 0: 3447
Class 1: 3504
Class 2: 3537


In [87]:
import os
from collections import Counter

def count_and_combine_classes(additional_dir, base_counts):
    # 1. 추가 프레임 폴더 확인
    if not os.path.exists(additional_dir):
        print(f"에러: 경로를 찾을 수 없습니다 -> {additional_dir}")
        return

    # 추가된 프레임에서의 카운트를 저장할 변수
    additional_counts = Counter()

    files = [f for f in os.listdir(additional_dir) if f.endswith('.txt')]
    print(f"'{additional_dir}' 폴더에서 {len(files)}개의 라벨 파일을 찾았습니다. 분석 중...")

    # 2. 텍스트 파일 파싱 및 카운팅
    for filename in files:
        filepath = os.path.join(additional_dir, filename)
        
        # 빈 파일 등 에러 처리
        try:
            with open(filepath, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) > 0:
                        # YOLO 포맷의 첫 번째 값은 Class ID
                        class_id = int(parts[0])
                        additional_counts[class_id] += 1
        except Exception as e:
            print(f"파일 읽기 오류 ({filename}): {e}")

    # 3. 결과 합산 및 출력
    print("\n" + "="*40)
    print(f"{'Class ID':<10} | {'Existing':<10} | {'Additional':<10} | {'Total':<10}")
    print("-" * 46)

    # 0, 1, 2 클래스에 대해 순서대로 출력 (데이터에 없는 클래스가 있을 수도 있으므로 union 사용)
    all_classes = sorted(set(base_counts.keys()) | set(additional_counts.keys()))

    total_counts = {}

    for cls in all_classes:
        existing = base_counts.get(cls, 0)
        added = additional_counts[cls]
        total = existing + added
        total_counts[cls] = total
        
        print(f"Class {cls:<4} | {existing:<10} | {added:<10} | {total:<10}")
    
    print("="*40)
    
    # 요약 정보
    print(f"\n[Additional Frames Summary]")
    for cls, count in sorted(additional_counts.items()):
        print(f" - Class {cls}: {count} objects")
        
    print(f"\n[Grand Total Summary]")
    for cls, count in sorted(total_counts.items()):
        print(f" - Class {cls}: {count} objects")

# --- 설정 ---

# 1. 추가할 프레임들이 있는 경로
target_dir = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/Euljiro_inner_20201128_f1038_t1519/other_tries/new_annotation_skku_30_frames_ing/train_balanced_orig/TrainVal_unbalanced/additional_frames'

# 2. 기존 2초 간격 프레임의 객체 수 (User가 제공한 값)
current_base_counts = {
    0: 3470,
    1: 2293,
    2: 2975
}

if __name__ == "__main__":
    count_and_combine_classes(target_dir, current_base_counts)

'/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/Euljiro_inner_20201128_f1038_t1519/other_tries/new_annotation_skku_30_frames_ing/train_balanced_orig/TrainVal_unbalanced/additional_frames' 폴더에서 1587개의 라벨 파일을 찾았습니다. 분석 중...

Class ID   | Existing   | Additional | Total     
----------------------------------------------
Class 0    | 3470       | 371        | 3841      
Class 1    | 2293       | 1529       | 3822      
Class 2    | 2975       | 866        | 3841      

[Additional Frames Summary]
 - Class 0: 371 objects
 - Class 1: 1529 objects
 - Class 2: 866 objects

[Grand Total Summary]
 - Class 0: 3841 objects
 - Class 1: 3822 objects
 - Class 2: 3841 objects


## extracting new frames.

In [48]:
import cv2
import pandas as pd
import os

def extract_frames_from_video():
    # --- 설정 영역 ---
    # 기본 경로 (비디오 파일과 CSV가 있는 루트 폴더)
    base_dir = '/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519'
    
    # CSV 파일 경로
    csv_path = os.path.join(base_dir, 'frame_intervals_filled.csv')
    
    # 추출된 이미지를 저장할 폴더 (새로 생성됨)
    output_dir = os.path.join(base_dir, 'extracted_new_frames')

    # 비디오 ID와 파일명 매핑
    video_map = {
        1: "1_2020-11-28_10-41-45.mp4",
        2: "2_2020-11-28_11-33-40.mp4",
        3: "3_2020-11-28_13-01-02.mp4",
        4: "4_2020-11-28_14-21-05.mp4",
        5: "5_2020-11-28_15-19-51.mp4"
    }
    # ----------------

    # 1. 출력 폴더 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"폴더 생성됨: {output_dir}")

    # 2. CSV 파일 읽기
    if not os.path.exists(csv_path):
        print(f"에러: CSV 파일을 찾을 수 없습니다 -> {csv_path}")
        return

    df = pd.read_csv(csv_path)
    
    # 'new' 상태인 프레임만 필터링
    new_frames_df = df[df['status'] == 'new']
    
    if new_frames_df.empty:
        print("추출할 'new' 프레임이 없습니다.")
        return

    print(f"총 {len(new_frames_df)}개의 새로운 프레임을 추출합니다...")

    # 3. 비디오별로 그룹화하여 처리 (비디오 로딩 횟수 최소화)
    for video_id, group in new_frames_df.groupby('video_id'):
        if video_id not in video_map:
            print(f"경고: video_id {video_id}에 해당하는 파일명이 매핑되지 않았습니다. 건너뜁니다.")
            continue

        video_filename = video_map[video_id]
        video_path = os.path.join(base_dir, video_filename)

        print(f"--- 처리 중: {video_filename} (ID: {video_id}) ---")

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"에러: 비디오를 열 수 없습니다 -> {video_path}")
            continue

        count = 0
        for _, row in group.iterrows():
            frame_num = int(row['frame'])
            
            # 해당 프레임으로 이동
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            ret, frame_img = cap.read()

            if ret:
                # 파일명 생성 (예: 1_007080.jpg) -> 6자리 숫자로 패딩
                output_filename = f"{video_id}_{frame_num:06d}.jpg"
                save_path = os.path.join(output_dir, output_filename)
                
                cv2.imwrite(save_path, frame_img)
                count += 1
            else:
                print(f"실패: 프레임 {frame_num}을 읽을 수 없습니다.")

        cap.release()
        print(f"  -> {count}장 추출 완료")

    print(f"\n모든 작업이 완료되었습니다. 결과물은 '{output_dir}' 폴더를 확인하세요.")

if __name__ == "__main__":
    extract_frames_from_video()

폴더 생성됨: /media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/extracted_new_frames
총 6452개의 새로운 프레임을 추출합니다...
--- 처리 중: 1_2020-11-28_10-41-45.mp4 (ID: 1) ---
  -> 1121장 추출 완료
--- 처리 중: 2_2020-11-28_11-33-40.mp4 (ID: 2) ---
  -> 1649장 추출 완료
--- 처리 중: 3_2020-11-28_13-01-02.mp4 (ID: 3) ---
  -> 1423장 추출 완료
--- 처리 중: 5_2020-11-28_15-19-51.mp4 (ID: 5) ---
  -> 2259장 추출 완료

모든 작업이 완료되었습니다. 결과물은 '/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/extracted_new_frames' 폴더를 확인하세요.


## Extracting missing video frames.

In [15]:
import pandas as pd
import cv2
import os

# ==========================================
# 1. Configuration
# ==========================================

# Path to your Excel/CSV file
excel_file_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro_inner_20201128_f1038_t1519/new_annotation_skku_30_frames/temp.xlsx'

# Path to the folder containing the MP4 video files
# (Based on your paths, I assume they are in the parent folder. Update this if they are elsewhere)
video_source_dir = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro_inner_20201128_f1038_t1519'

# Output directory for extracted frames
output_dir = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro_inner_20201128_f1038_t1519/new_annotation_skku_30_frames/extracted_frame'

# Map Video ID to Filenames
video_map = {
    1: "/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/1_2020-11-28_10-41-45.mp4",
    2: "/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/2_2020-11-28_11-33-40.mp4",
    3: "/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/3_2020-11-28_13-01-02.mp4",
    4: "/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/4_2020-11-28_14-21-05.mp4",
    5: "/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/5_2020-11-28_15-19-51.mp4"
}

# ==========================================
# 2. Setup
# ==========================================
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created output directory: {output_dir}")

print(f"Loading annotation file: {excel_file_path}")

# Load Data
try:
    if excel_file_path.endswith('.csv'):
        df = pd.read_csv(excel_file_path)
    else:
        df = pd.read_excel(excel_file_path)
except Exception as e:
    print(f"Error loading file: {e}")
    exit()

# Clean column names
df.columns = df.columns.str.strip().str.lower()
df['status'] = df['status'].astype(str).str.strip().str.lower()

# Filter for "new extraction"
# (Checking for both "new extraction" and "new_extraction" just in case)
extract_df = df[df['status'].isin(['new extraction', 'new_extraction'])]

print(f"Found {len(extract_df)} frames to extract.")

# ==========================================
# 3. Extraction Loop
# ==========================================
success_count = 0
error_count = 0

# We'll open video captures as needed to be efficient, or open/close per file.
# Since frames might be scattered, opening/closing per frame is slow. 
# Better: Group by video_id.

for vid_id, group in extract_df.groupby('video_id'):
    vid_id = int(vid_id)
    
    if vid_id not in video_map:
        print(f"Warning: Video ID {vid_id} not defined in video_map. Skipping.")
        continue
        
    video_filename = video_map[vid_id]
    video_path = os.path.join(video_source_dir, video_filename)
    
    if not os.path.exists(video_path):
        print(f"Error: Video file not found: {video_path}")
        continue
        
    print(f"Processing Video {vid_id}: {video_filename} ...")
    
    # Open Video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_filename}")
        continue
        
    # Iterate through frames for this video
    for _, row in group.iterrows():
        frame_num = int(row['frame_num'])
        
        # Construct output filename: e.g., 1_000060.jpg
        output_filename = f"{vid_id}_{frame_num:06d}.jpg"
        output_path = os.path.join(output_dir, output_filename)
        
        try:
            # Set frame position (0-based index)
            # If your frame numbers are 1-based, use (frame_num - 1)
            # Assuming 0-based based on standard CV practices.
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            
            ret, frame = cap.read()
            
            if ret:
                cv2.imwrite(output_path, frame)
                # print(f"Saved: {output_filename}") # Uncomment for verbose output
                success_count += 1
            else:
                print(f"Error: Could not read frame {frame_num} from {video_filename}")
                error_count += 1
                
        except Exception as e:
            print(f"Exception extracting {output_filename}: {e}")
            error_count += 1
            
    cap.release()

# ==========================================
# 4. Summary
# ==========================================
print("-" * 30)
print(f"Extraction Complete.")
print(f"Frames extracted: {success_count}")
print(f"Errors: {error_count}")
print(f"Saved to: {output_dir}")
print("-" * 30)

Loading annotation file: /media/holidayj/Documents/Data/Platform/final_dataset/Euljiro_inner_20201128_f1038_t1519/new_annotation_skku_30_frames/temp.xlsx
Found 2 frames to extract.
Processing Video 1: /media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/1_2020-11-28_10-41-45.mp4 ...
Processing Video 2: /media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/2_2020-11-28_11-33-40.mp4 ...
------------------------------
Extraction Complete.
Frames extracted: 2
Errors: 0
Saved to: /media/holidayj/Documents/Data/Platform/final_dataset/Euljiro_inner_20201128_f1038_t1519/new_annotation_skku_30_frames/extracted_frame
------------------------------


## Template matching.

In [16]:
import cv2
import numpy as np
import os

# ==========================================
# Configuration
# ==========================================
# Path to the HD (Original) Image
hd_image_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro_inner_20201128_f1038_t1519/new_annotation_skku_30_frames/extracted_frame/2_116880_hd.jpg'

# Path to the Cropped Image
# (I adjusted the path slightly based on your likely folder structure, please verify)
crop_image_path = '/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro_inner_20201128_f1038_t1519/new_annotation_skku_30_frames/extracted_frame/2_116880.jpg'

# ==========================================
# Processing
# ==========================================
print(f"Loading HD Image: {hd_image_path}")
print(f"Loading Crop Image: {crop_image_path}")

# 1. Load images
img_hd = cv2.imread(hd_image_path)
img_crop = cv2.imread(crop_image_path)

if img_hd is None or img_crop is None:
    print("Error: Could not load one or both images. Check file paths.")
else:
    # 2. Perform Template Matching
    # TM_CCOEFF_NORMED is robust for finding exact matches
    result = cv2.matchTemplate(img_hd, img_crop, cv2.TM_CCOEFF_NORMED)
    
    # 3. Find the location of the best match
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    
    # max_loc gives the top-left (x, y) of the best match
    top_left = max_loc
    confidence = max_val
    
    # 4. Calculate Bottom-Right
    h, w = img_crop.shape[:2]
    bottom_right = (top_left[0] + w, top_left[1] + h)
    
    print("-" * 30)
    print(f"Match Confidence: {confidence:.5f} (1.0 is a perfect match)")
    
    if confidence > 0.99:
        print(f"FOUND EXACT LOCATION!")
        print(f"Top-Left (x, y)     : {top_left}")
        print(f"Bottom-Right (x, y) : {bottom_right}")
        print(f"Crop Width/Height   : {w} x {h}")
        
        # Python Slicing Format: [y1:y2, x1:x2]
        print(f"\nPython Crop Code Hint:")
        print(f"cropped = original[{top_left[1]}:{bottom_right[1]}, {top_left[0]}:{bottom_right[0]}]")
    else:
        print("Warning: Match confidence is low. The crop might be resized or modified.")
    print("-" * 30)

Loading HD Image: /media/holidayj/Documents/Data/Platform/final_dataset/Euljiro_inner_20201128_f1038_t1519/new_annotation_skku_30_frames/extracted_frame/2_116880_hd.jpg
Loading Crop Image: /media/holidayj/Documents/Data/Platform/final_dataset/Euljiro_inner_20201128_f1038_t1519/new_annotation_skku_30_frames/extracted_frame/2_116880.jpg
------------------------------
Match Confidence: 0.99912 (1.0 is a perfect match)
FOUND EXACT LOCATION!
Top-Left (x, y)     : (1327, 120)
Bottom-Right (x, y) : (1647, 440)
Crop Width/Height   : 320 x 320

Python Crop Code Hint:
cropped = original[120:440, 1327:1647]
------------------------------


## Crop hd frame to 320*320

In [49]:
import cv2
import os

# ==========================================
# 1. Configuration
# ==========================================
# Input folder containing the full HD frames
source_dir = '/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/extracted_new_frames'

# Output folder for the cropped images
output_dir = os.path.join(source_dir, 'cropped')

# Crop Coordinates (Found from previous step)
# Slice format: [y_start : y_end, x_start : x_end]
# Top-Left (x, y): (1327, 120)
# Bottom-Right (x, y): (1647, 440)
y_start, y_end = 120, 440
x_start, x_end = 1327, 1647

# ==========================================
# 2. Processing
# ==========================================
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created output directory: {output_dir}")

print(f"Source Directory: {source_dir}")
print(f"Crop Target: y[{y_start}:{y_end}], x[{x_start}:{x_end}]")

processed_count = 0
error_count = 0

for filename in os.listdir(source_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        
        file_path = os.path.join(source_dir, filename)
        
        # Read the image
        img = cv2.imread(file_path)
        
        if img is None:
            print(f"Error: Could not read {filename}")
            error_count += 1
            continue
        
        # Check if image is large enough to crop
        h, w = img.shape[:2]
        if h < y_end or w < x_end:
            print(f"Skipping {filename}: Image too small ({w}x{h}) for crop coordinates.")
            error_count += 1
            continue
            
        # Perform Crop
        cropped_img = img[y_start:y_end, x_start:x_end]
        
        # Save
        save_path = os.path.join(output_dir, filename)
        cv2.imwrite(save_path, cropped_img)
        
        processed_count += 1

# ==========================================
# 3. Summary
# ==========================================
print("-" * 30)
print(f"Processing Complete.")
print(f"Images Cropped: {processed_count}")
print(f"Errors/Skipped: {error_count}")
print(f"Saved to: {output_dir}")
print("-" * 30)

Created output directory: /media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/extracted_new_frames/cropped
Source Directory: /media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/extracted_new_frames
Crop Target: y[120:440], x[1327:1647]
------------------------------
Processing Complete.
Images Cropped: 6452
Errors/Skipped: 0
Saved to: /media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/extracted_new_frames/cropped
------------------------------


## Crop and save passengers

In [3]:
import os
import cv2
import numpy as np
import glob
from tqdm import tqdm

# --- CONFIGURATION ---
SOURCE_DIR = "/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/Euljiro_off_peak_TrainVal_unbalanced_2sec_FINAL_20260117/TrainVal_combined"
OUTPUT_DIR = "/media/holidayj/Documents/Data/Platform/final_dataset/cropped_passengers"

# Margin Ratio (0.2 = 20% padding on each side)
MARGIN_RATIO = 0.1
TARGET_SIZE = (224, 224)

# --- CORRECTED CLASS MAPPING ---
# You must check your 'classes.txt' file to see which ID belongs to which name.
# I am assuming 0=Up, 1=Down, 2=Pass based on your description.
# If your order is different, swap the numbers here!
CLASS_MAP = {
    0: 'U',  # Up
    1: 'D',  # Down
    2: 'P',  # Pass
}

def safe_crop_with_padding(image, box_coords, target_size=(224, 224)):
    """
    Crops the image with zero-padding (black borders) for out-of-bound areas.
    box_coords: [x1, y1, x2, y2] (Absolute pixel values, can be outside image)
    """
    h_img, w_img, _ = image.shape
    x1, y1, x2, y2 = box_coords
    
    # 1. Calculate desired width/height
    cw, ch = x2 - x1, y2 - y1
    
    # 2. Create black canvas
    canvas = np.zeros((ch, cw, 3), dtype=image.dtype)
    
    # 3. Calculate intersection with actual image
    x1_valid = max(0, x1)
    y1_valid = max(0, y1)
    x2_valid = min(w_img, x2)
    y2_valid = min(h_img, y2)
    
    # 4. Paste valid intersection onto canvas
    if x1_valid < x2_valid and y1_valid < y2_valid:
        crop_valid = image[y1_valid:y2_valid, x1_valid:x2_valid]
        
        # Calculate offsets on canvas
        off_x = x1_valid - x1
        off_y = y1_valid - y1
        
        canvas[off_y : off_y + crop_valid.shape[0], 
               off_x : off_x + crop_valid.shape[1]] = crop_valid
               
    # 5. Resize to standard CNN input size
    return cv2.resize(canvas, target_size)

def process_dataset():
    # Create output subfolders
    for class_name in set(CLASS_MAP.values()):
        os.makedirs(os.path.join(OUTPUT_DIR, class_name), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, "U"), exist_ok=True) # Fallback folder

    # Get list of text files
    txt_files = glob.glob(os.path.join(SOURCE_DIR, "*.txt"))
    
    print(f"Found {len(txt_files)} label files. Starting processing...")

    for txt_path in tqdm(txt_files):
        # 1. Parse filename to get ID and Frame
        base_name = os.path.basename(txt_path)
        file_root = os.path.splitext(base_name)[0] # e.g., "5_179640"
        
        # Find matching image (try jpg, then png)
        img_path = os.path.join(SOURCE_DIR, file_root + ".jpg")
        if not os.path.exists(img_path):
            img_path = os.path.join(SOURCE_DIR, file_root + ".png")
            if not os.path.exists(img_path):
                continue # Skip if no image found

        # 2. Load Image
        img = cv2.imread(img_path)
        if img is None: continue
        h_img, w_img, _ = img.shape

        # 3. Read Labels
        with open(txt_path, 'r') as f:
            lines = f.readlines()

        # 4. Process each object
        for idx, line in enumerate(lines):
            parts = line.strip().split()
            if len(parts) < 5: continue
            
            cls_id = int(parts[0])
            # YOLO format: x_center, y_center, width, height (Normalized 0-1)
            n_xc, n_yc, n_w, n_h = map(float, parts[1:5])
            
            # Convert to Pixel Coordinates (x1, y1, x2, y2)
            w_box = int(n_w * w_img)
            h_box = int(n_h * h_img)
            x_center = int(n_xc * w_img)
            y_center = int(n_yc * h_img)
            
            x1 = x_center - w_box // 2
            y1 = y_center - h_box // 2
            x2 = x1 + w_box
            y2 = y1 + h_box
            
            # 5. Apply Safe Margin (Padding)
            pad_w = int(w_box * MARGIN_RATIO)
            pad_h = int(h_box * MARGIN_RATIO)
            
            # Coordinates can go negative or exceed image size (Safe Crop handles this)
            crop_coords = [x1 - pad_w, y1 - pad_h, x2 + pad_w, y2 + pad_h]
            
            # 6. Crop
            cropped_img = safe_crop_with_padding(img, crop_coords, TARGET_SIZE)
            
            # 7. Generate Filename
            # Format: {video_id}_{frame}_{Class}_{Index}.jpg
            # Index is 0-padded (e.g., 00, 01, 02)
            cls_name = CLASS_MAP.get(cls_id, 'U')
            obj_index = f"{idx:02d}" 
            
            save_name = f"{file_root}_{cls_name}_{obj_index}.jpg"
            save_path = os.path.join(OUTPUT_DIR, cls_name, save_name)
            
            # 8. Save
            cv2.imwrite(save_path, cropped_img)

if __name__ == "__main__":
    process_dataset()

Found 3447 label files. Starting processing...


100%|██████████| 3447/3447 [00:14<00:00, 239.41it/s]


## Extracting frames t-1 and t-2

In [10]:
import os
import cv2
import glob
from tqdm import tqdm

# --- CONFIGURATION ---
SOURCE_DIR = "/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/Euljiro_off_peak_TrainVal_unbalanced_2sec_FINAL_20260117/TrainVal_combined"
OUTPUT_DIR = "/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/extracted_frames_t_t1_t2"

VIDEO_PATHS = {
    1: "/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/1_2020-11-28_10-41-45.mp4",
    2: "/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/2_2020-11-28_11-33-40.mp4",
    3: "/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/3_2020-11-28_13-01-02.mp4",
    4: "/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/4_2020-11-28_14-21-05.mp4",
    5: "/media/holidayj/Documents/Data/Platform/Euljiro/Euljiro_inner_20201128_f1038_t1519/5_2020-11-28_15-19-51.mp4"
}

def extract_all_three_frames():
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # 1. Parse all filenames from SOURCE_DIR to find targets
    jpg_files = glob.glob(os.path.join(SOURCE_DIR, "*.jpg"))
    print(f"Found {len(jpg_files)} source images. Building task list...")

    tasks = []
    for jpg_path in jpg_files:
        base_name = os.path.basename(jpg_path)
        file_root = os.path.splitext(base_name)[0]
        try:
            parts = file_root.split('_')
            vid_id = int(parts[0])
            frame_num = int(parts[1])
            tasks.append({'vid_id': vid_id, 'frame_num': frame_num})
        except (ValueError, IndexError):
            continue

    # 2. Sort by Video -> Frame (Optimization)
    tasks.sort(key=lambda x: (x['vid_id'], x['frame_num']))

    current_vid_id = None
    cap = None
    
    print(f"Processing {len(tasks)} sequences (3 frames each)...")
    
    for i in tqdm(range(len(tasks))):
        t = tasks[i]
        vid_id = t['vid_id']
        target_frame = t['frame_num']
        
        # Calculate Frame Numbers
        f_t2 = max(0, target_frame - 2)
        f_t1 = max(0, target_frame - 1)
        f_t0 = target_frame 
        
        frames_to_extract = [f_t2, f_t1, f_t0]
        
        # Check if they already exist
        all_exist = True
        for f_num in frames_to_extract:
            path = os.path.join(OUTPUT_DIR, f"{vid_id}_{f_num:06d}.jpg")
            if not os.path.exists(path):
                all_exist = False
                break
        
        if all_exist:
            continue

        # Open Video if changed
        if vid_id != current_vid_id:
            if cap is not None: cap.release()
            video_path = VIDEO_PATHS.get(vid_id)
            if video_path is None: continue
            cap = cv2.VideoCapture(video_path)
            current_vid_id = vid_id
        
        if cap is None or not cap.isOpened(): continue

        # --- SMART SEEK ---
        # We start reading at T-2
        start_frame = frames_to_extract[0]
        current_pos = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        
        dist = start_frame - current_pos
        
        # If we are close (0 to 20 frames), read forward. Otherwise seek.
        if 0 <= dist < 20:
            for _ in range(dist):
                cap.read()
        else:
            cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        # --- EXTRACT 3 CONSECUTIVE FRAMES ---
        # Read T-2, T-1, T0 sequentially
        for f_num in frames_to_extract:
            ret, frame = cap.read()
            if ret:
                save_path = os.path.join(OUTPUT_DIR, f"{vid_id}_{f_num:06d}.jpg")
                # Only save if we strictly need it (skip existing to save IO)
                if not os.path.exists(save_path):
                    cv2.imwrite(save_path, frame)
            else:
                break # Stop if video ends

    if cap is not None: cap.release()
    print("Extraction Complete: All 3 frames (t, t-1, t-2) saved.")

if __name__ == "__main__":
    extract_all_three_frames()

Found 3447 source images. Building task list...
Processing 3447 sequences (3 frames each)...


  0%|          | 0/3447 [00:00<?, ?it/s]

100%|██████████| 3447/3447 [13:54<00:00,  4.13it/s]

Extraction Complete: All 3 frames (t, t-1, t-2) saved.


## Crop the frames.

In [11]:
import os
import cv2
import glob
from tqdm import tqdm

# --- CONFIGURATION ---
# Source: The folder where we saved the Full HD t-1 and t-2 frames in Step 1
SOURCE_DIR = "/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/extracted_frames_t_t1_t2"

# Output: Where to save the 320x320 ROI cropped frames
OUTPUT_DIR = "/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/roi_frames_320x320"

# ROI Coordinates
Y_START, Y_END = 120, 440
X_START, X_END = 1327, 1647

def crop_roi_frames():
    # Create output directory
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Get all jpg files from the source directory
    # This includes both _t1.jpg and _t2.jpg files
    img_files = glob.glob(os.path.join(SOURCE_DIR, "*.jpg"))
    
    print(f"Found {len(img_files)} frames to crop. Starting ROI processing...")
    
    for img_path in tqdm(img_files):
        # 1. Read the Full HD Image
        img = cv2.imread(img_path)
        
        if img is None:
            print(f"Warning: Could not read {img_path}")
            continue

        # 2. Apply Fixed ROI Crop
        # Note: numpy slicing is [y:y_end, x:x_end]
        roi_crop = img[Y_START:Y_END, X_START:X_END]
        
        # Optional validation: Check if size is correct (320x320)
        if roi_crop.shape[0] != (Y_END - Y_START) or roi_crop.shape[1] != (X_END - X_START):
            print(f"Warning: Crop size mismatch for {img_path}. Check coordinates.")
        
        # 3. Save to Output Directory
        # We keep the same filename (e.g., 5_179640_t1.jpg)
        filename = os.path.basename(img_path)
        save_path = os.path.join(OUTPUT_DIR, filename)
        
        cv2.imwrite(save_path, roi_crop)

    print("Step 2 Complete. All frames cropped to ROI (320x320).")

if __name__ == "__main__":
    crop_roi_frames()

Found 10339 frames to crop. Starting ROI processing...


  0%|          | 0/10339 [00:00<?, ?it/s]

100%|██████████| 10339/10339 [04:45<00:00, 36.24it/s]

Step 2 Complete. All frames cropped to ROI (320x320).


## Crop passenger object

In [12]:
import os
import cv2
import numpy as np
import glob
from tqdm import tqdm

# --- CONFIGURATION ---
# 1. Where your labeled text files and images are located
SOURCE_DIR = "/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/roi_frames_320x320"

# 2. Where the extracted full frames (from the previous step) are located
# We need to look here to find the t-1 and t-2 images.
EXTRACTED_FRAMES_DIR = "/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/roi_frames_320x320"

# 3. Where to save the final cropped passenger triplets
OUTPUT_DIR = "/media/holidayj/Documents/Data/Platform/final_dataset/cropped_passengers_temporal"

MARGIN_RATIO = 0.1
CLASS_MAP = {0: 'U', 1: 'D', 2: 'P'}

def safe_crop_with_padding(image, box_coords):
    h_img, w_img, _ = image.shape
    x1, y1, x2, y2 = box_coords
    cw, ch = x2 - x1, y2 - y1
    canvas = np.zeros((ch, cw, 3), dtype=image.dtype)
    
    x1_valid = max(0, x1)
    y1_valid = max(0, y1)
    x2_valid = min(w_img, x2)
    y2_valid = min(h_img, y2)
    
    if x1_valid < x2_valid and y1_valid < y2_valid:
        crop_valid = image[y1_valid:y2_valid, x1_valid:x2_valid]
        off_x = x1_valid - x1
        off_y = y1_valid - y1
        canvas[off_y : off_y + crop_valid.shape[0], off_x : off_x + crop_valid.shape[1]] = crop_valid
               
    return canvas

def process_temporal_crops():
    # Create output subfolders
    for class_name in set(CLASS_MAP.values()):
        os.makedirs(os.path.join(OUTPUT_DIR, class_name), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, "U"), exist_ok=True)

    # We iterate based on the LABEL files (Frame t)
    txt_files = glob.glob(os.path.join(SOURCE_DIR, "*.txt"))
    
    print(f"Found {len(txt_files)} labels. Generating temporal crops...")

    for txt_path in tqdm(txt_files):
        # 1. Identify Frame t info
        base_name = os.path.basename(txt_path)
        file_root = os.path.splitext(base_name)[0]
        
        try:
            parts = file_root.split('_')
            vid_id = int(parts[0])
            frame_t = int(parts[1])
        except:
            continue

        # 2. Determine filenames for t, t-1, t-2
        # We look for these in the EXTRACTED_FRAMES_DIR we created earlier
        # Format: {vid_id}_{frame:06d}.jpg
        fname_t0 = f"{vid_id}_{frame_t:06d}.jpg"
        fname_t1 = f"{vid_id}_{frame_t-1:06d}.jpg"
        fname_t2 = f"{vid_id}_{frame_t-2:06d}.jpg"
        
        path_t0 = os.path.join(EXTRACTED_FRAMES_DIR, fname_t0)
        path_t1 = os.path.join(EXTRACTED_FRAMES_DIR, fname_t1)
        path_t2 = os.path.join(EXTRACTED_FRAMES_DIR, fname_t2)

        # Skip if any frame is missing (e.g. at the very start of a video)
        if not (os.path.exists(path_t0) and os.path.exists(path_t1) and os.path.exists(path_t2)):
            continue

        # 3. Load Images
        img_t0 = cv2.imread(path_t0)
        img_t1 = cv2.imread(path_t1)
        img_t2 = cv2.imread(path_t2)
        
        if img_t0 is None: continue
        h_img, w_img, _ = img_t0.shape

        # 4. Read Labels (from Frame t)
        with open(txt_path, 'r') as f:
            lines = f.readlines()

        # 5. Process each passenger
        for idx, line in enumerate(lines):
            parts = line.strip().split()
            if len(parts) < 5: continue
            
            cls_id = int(parts[0])
            n_xc, n_yc, n_w, n_h = map(float, parts[1:5])
            
            # --- CALCULATE BOX (Based on Frame t) ---
            w_box = int(n_w * w_img)
            h_box = int(n_h * h_img)
            x_center = int(n_xc * w_img)
            y_center = int(n_yc * h_img)
            
            x1 = x_center - w_box // 2
            y1 = y_center - h_box // 2
            x2 = x1 + w_box
            y2 = y1 + h_box
            
            # Apply Margin
            pad_w = int(w_box * MARGIN_RATIO)
            pad_h = int(h_box * MARGIN_RATIO)
            crop_coords = [x1 - pad_w, y1 - pad_h, x2 + pad_w, y2 + pad_h]
            
            # --- APPLY SAME CROP TO ALL 3 FRAMES ---
            crop_0 = safe_crop_with_padding(img_t0, crop_coords)
            crop_1 = safe_crop_with_padding(img_t1, crop_coords)
            crop_2 = safe_crop_with_padding(img_t2, crop_coords)
            
            # --- SAVE ---
            cls_name = CLASS_MAP.get(cls_id, 'U')
            obj_index = f"{idx:02d}"
            
            # Save t0 (Current)
            name_0 = f"{vid_id}_{frame_t}_{cls_name}_{obj_index}_t0.jpg"
            cv2.imwrite(os.path.join(OUTPUT_DIR, cls_name, name_0), crop_0)

            # Save t1 (Previous)
            name_1 = f"{vid_id}_{frame_t}_{cls_name}_{obj_index}_t1.jpg"
            cv2.imwrite(os.path.join(OUTPUT_DIR, cls_name, name_1), crop_1)

            # Save t2 (2 frames ago)
            name_2 = f"{vid_id}_{frame_t}_{cls_name}_{obj_index}_t2.jpg"
            cv2.imwrite(os.path.join(OUTPUT_DIR, cls_name, name_2), crop_2)

if __name__ == "__main__":
    process_temporal_crops()

Found 3447 labels. Generating temporal crops...


100%|██████████| 3447/3447 [00:24<00:00, 138.08it/s]


## Converting 3-class to 2-class.

In [4]:
import os
import glob
from tqdm import tqdm

def convert_3class_to_2class(input_folder, output_folder):
    """
    Converts YOLO annotations from 3 classes to 2 classes.
    Mapping:
        Original 0 -> New 0
        Original 2 -> New 0
        Original 1 -> New 1
    """
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Get all txt files
    txt_files = glob.glob(os.path.join(input_folder, "*.txt"))
    print(f"Found {len(txt_files)} files in {input_folder}")
    
    for txt_file in tqdm(txt_files, desc="Converting files"):
        filename = os.path.basename(txt_file)
        output_path = os.path.join(output_folder, filename)
        
        new_lines = []
        
        with open(txt_file, 'r') as f:
            lines = f.readlines()
            
        for line in lines:
            parts = line.strip().split()
            if not parts:
                continue
                
            # Parse original class
            cls_id = int(parts[0])
            coords = parts[1:] # x, y, w, h
            
            # --- MAPPING LOGIC ---
            # Original 0 and 2 become 0
            # Original 1 remains 1
            if cls_id == 0 or cls_id == 2:
                new_cls_id = 0
            elif cls_id == 1:
                new_cls_id = 1
            else:
                # Handle unexpected classes if necessary (e.g., skip or keep)
                # print(f"Warning: Unexpected class {cls_id} in {filename}")
                continue 
            
            # Reconstruct the line
            new_line = f"{new_cls_id} {' '.join(coords)}\n"
            new_lines.append(new_line)
            
        # Write to new file in output folder
        with open(output_path, 'w') as f_out:
            f_out.writelines(new_lines)

if __name__ == "__main__":
    # --- CONFIGURATION ---
    # The path you provided
    input_dir = "/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/2_class/Euljiro_off_peak_TrainVal_balanced_2sec_plus_addition_1sec_FINAL_20260117/val"
    
    # Suggested output path (creates a 'converted' folder next to the original)
    # You can change this path if you want it somewhere else
    output_dir = os.path.join(input_dir, "converted_2class_labels")
    
    print(f"Processing from: {input_dir}")
    print(f"Saving to:     {output_dir}")
    
    convert_3class_to_2class(input_dir, output_dir)
    print("Done! Please check the 'converted_2class_labels' folder.")

Processing from: /media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/2_class/Euljiro_off_peak_TrainVal_balanced_2sec_plus_addition_1sec_FINAL_20260117/val
Saving to:     /media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/2_class/Euljiro_off_peak_TrainVal_balanced_2sec_plus_addition_1sec_FINAL_20260117/val/converted_2class_labels
Found 518 files in /media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/2_class/Euljiro_off_peak_TrainVal_balanced_2sec_plus_addition_1sec_FINAL_20260117/val


Converting files: 100%|██████████| 518/518 [00:00<00:00, 4464.69it/s]

Done! Please check the 'converted_2class_labels' folder.


## To give perturbation to the test set.

In [3]:
import cv2
import numpy as np
import random
import os
import glob
from pathlib import Path
from tqdm import tqdm

# ================= CONFIGURATION =================
# Input directory
INPUT_DIR = "/media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/3_class/Euljiro_off_peak_Test"

# Output directory (Will be created automatically)
OUTPUT_DIR = INPUT_DIR + "_Perturbed_BlackBorder"

# Perturbation Parameters (Slight changes as per reviewer)
ANGLE_LIMIT = 3       # +/- 5 degrees
SHIFT_LIMIT = 0.03    # +/- 5% shift
SCALE_LIMIT = 0.03    # +/- 5% scale (0.95 - 1.05)
BRIGHTNESS_LIMIT = 5 # +/- 10 pixel values
# =================================================

def yolo_to_corners(yolo_line, img_w, img_h):
    """Convert single YOLO line to [xmin, ymin, xmax, ymax]"""
    parts = list(map(float, yolo_line.split()))
    cls_id = int(parts[0])
    cx, cy, w, h = parts[1], parts[2], parts[3], parts[4]
    
    x_min = int((cx - w/2) * img_w)
    y_min = int((cy - h/2) * img_h)
    x_max = int((cx + w/2) * img_w)
    y_max = int((cy + h/2) * img_h)
    
    return cls_id, [x_min, y_min, x_max, y_max]

def corners_to_yolo(box, img_w, img_h):
    """Convert [xmin, ymin, xmax, ymax] back to YOLO [cx, cy, w, h] normalized"""
    xmin, ymin, xmax, ymax = box
    
    # Clip to image boundaries
    xmin = max(0, xmin); ymin = max(0, ymin)
    xmax = min(img_w, xmax); ymax = min(img_h, ymax)
    
    # Calculate width/height
    box_w = xmax - xmin
    box_h = ymax - ymin
    
    # Avoid zero-area boxes
    if box_w <= 1 or box_h <= 1:
        return None
        
    cx = (xmin + box_w/2.0) / img_w
    cy = (ymin + box_h/2.0) / img_h
    nw = box_w / float(img_w)
    nh = box_h / float(img_h)
    
    return [cx, cy, nw, nh]

def apply_perturbation(image, boxes, angle_lim, shift_lim, scale_lim, bright_lim):
    h, w = image.shape[:2]
    cx, cy = w // 2, h // 2

    # 1. Random Parameters
    angle = random.uniform(-angle_lim, angle_lim)
    tx = random.uniform(-shift_lim, shift_lim) * w
    ty = random.uniform(-shift_lim, shift_lim) * h
    scale = random.uniform(1 - scale_lim, 1 + scale_lim)
    beta = random.uniform(-bright_lim, bright_lim) 

    # 2. Geometric Transform (Rotation + Scale + Translation)
    M = cv2.getRotationMatrix2D((cx, cy), angle, scale)
    M[0, 2] += tx
    M[1, 2] += ty

    # === UPDATED SECTION: Black Borders ===
    # borderMode=cv2.BORDER_CONSTANT fills outer area with a solid color
    # borderValue=(0, 0, 0) specifies that color is black
    aug_image = cv2.warpAffine(
        image, M, (w, h), 
        borderMode=cv2.BORDER_CONSTANT, 
        borderValue=(0, 0, 0)
    )
    # ======================================
    
    # Apply Brightness
    aug_image = cv2.convertScaleAbs(aug_image, alpha=1.0, beta=beta)

    # 3. Transform Boxes
    aug_boxes = [] 
    
    for cls_id, box in boxes:
        x_min, y_min, x_max, y_max = box
        
        corners = np.array([
            [x_min, y_min], [x_max, y_min],
            [x_max, y_max], [x_min, y_max]
        ])
        ones = np.ones((4, 1))
        corners_ones = np.hstack([corners, ones])
        
        # Transform corners
        tf_corners = M.dot(corners_ones.T).T
        
        # New axis-aligned box
        new_xmin = np.min(tf_corners[:, 0])
        new_ymin = np.min(tf_corners[:, 1])
        new_xmax = np.max(tf_corners[:, 0])
        new_ymax = np.max(tf_corners[:, 1])
        
        # Convert back to YOLO
        yolo_box = corners_to_yolo([new_xmin, new_ymin, new_xmax, new_ymax], w, h)
        
        if yolo_box is not None:
            aug_boxes.append((cls_id, yolo_box))
            
    return aug_image, aug_boxes

def main():
    # Setup Paths
    input_path = Path(INPUT_DIR)
    output_path = Path(OUTPUT_DIR)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Get all images
    img_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    img_files = []
    for ext in img_extensions:
        img_files.extend(list(input_path.glob(ext)))
    
    print(f"Found {len(img_files)} images in {INPUT_DIR}")
    print(f"Processing to {OUTPUT_DIR}...")

    for img_file in tqdm(img_files):
        # 1. Read Image
        img = cv2.imread(str(img_file))
        if img is None: continue
        h, w = img.shape[:2]
        
        # 2. Read Label
        label_file = img_file.with_suffix('.txt')
        boxes = [] 
        
        if label_file.exists():
            with open(label_file, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    if line.strip():
                        cls_id, pixel_box = yolo_to_corners(line, w, h)
                        boxes.append((cls_id, pixel_box))
        
        # 3. Apply Perturbation
        aug_img, aug_yolo_data = apply_perturbation(
            img, boxes, ANGLE_LIMIT, SHIFT_LIMIT, SCALE_LIMIT, BRIGHTNESS_LIMIT
        )
        
        # 4. Save Image
        out_img_path = output_path / img_file.name
        cv2.imwrite(str(out_img_path), aug_img)
        
        # 5. Save Label
        out_lbl_path = output_path / label_file.name
        with open(out_lbl_path, 'w') as f:
            for cls_id, (cx, cy, nw, nh) in aug_yolo_data:
                f.write(f"{cls_id} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}\n")

    print(f"\nDone! Perturbed dataset saved to: {OUTPUT_DIR}")

if __name__ == "__main__":
    main()

Found 536 images in /media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/3_class/Euljiro_off_peak_Test
Processing to /media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/3_class/Euljiro_off_peak_Test_Perturbed_BlackBorder...


100%|██████████| 536/536 [00:03<00:00, 174.55it/s]


Done! Perturbed dataset saved to: /media/holidayj/Documents/Data/Platform/final_dataset/Euljiro/0_2_Original_dataset_Euljiro_off_peak_inner_20201128_f1038_t1519/3_class/Euljiro_off_peak_Test_Perturbed_BlackBorder
